In [ ]:
import random
import numpy as np
from PIL import Image, ImageDraw,ImageFont
import os
import math
from shutil import rmtree
import time
import openpyxl

In [ ]:
def do_point_map(): #Создания случайных точек на плоскости
  for now in range(nCity):
    point_x.append(random.randint(1, window_x-2))
    point_y.append(random.randint(1, window_y-2))
    for other in range(now):
      dist=((point_x[other]-point_x[now])**2 + (point_y[other]-point_y[now])**2)**0.5
      lenTrail[now][other]=dist
      lenTrail[other][now]=dist

In [ ]:
def FirstDraw(): # Рисование ранее построенного графа
  image = Image.new("RGB", (window_x*ScrCoef,window_y*ScrCoef), (255,255,255))
  draw = ImageDraw.Draw(image)
  for now in range(nCity):
    for other in range(now):
      draw.line([(ScrCoef*point_x[other],ScrCoef*point_y[other]),(ScrCoef*point_x[now],ScrCoef*point_y[now])], fill ="black", width = 0) 
  for now in range(nCity):
    if(now==start or now==end):
      draw.ellipse([(ScrCoef*point_x[now]-5,ScrCoef*point_y[now]-5),(ScrCoef*point_x[now]+5,ScrCoef*point_y[now]+5)], fill = 'blue', outline ='blue')
      draw.text((ScrCoef*point_x[now]+5,ScrCoef*point_y[now]+5), str(now),fill='blue')
    else:
      draw.ellipse([(ScrCoef*point_x[now]-2,ScrCoef*point_y[now]-2),(ScrCoef*point_x[now]+2,ScrCoef*point_y[now]+2)], fill = 'red', outline ='red')
      draw.text((ScrCoef*point_x[now]+2,ScrCoef*point_y[now]+2), str(now),fill='red')
  del draw
  image.save("Map.png", "PNG")

In [ ]:
def ProgressDraw(time,bestLen,Trail,FoldName): #Рисование карты - промежуточно найденного лучшего пути
  image = Image.new("RGB", (ScrCoef*window_x,ScrCoef*window_y), (255,255,255))
  draw = ImageDraw.Draw(image)
  if(AllLines==1):
    for now in range(nCity):
      for other in range(now):
        draw.line([(ScrCoef*point_x[now],ScrCoef*point_y[now]),(ScrCoef*point_x[other],ScrCoef*point_y[other])], fill ="black", width = 2)
    
    for now in range(nCity):
      for other in range(now):
        draw.text((ScrCoef*(point_x[other]+point_x[now])/2-2,ScrCoef*(point_y[other]+point_y[now])/2-2), str('%.3f' % pherTrail[now][other]),fill='blue')

  
  for i in range(len(Trail)-1):
     draw.line([(ScrCoef*point_x[Trail[i]],ScrCoef*point_y[Trail[i]]),(ScrCoef*point_x[Trail[i+1]],ScrCoef*point_y[Trail[i+1]])], fill ="red", width = 5)
  for now in range(nCity):
    draw.ellipse([(ScrCoef*point_x[now]-2,ScrCoef*point_y[now]-2),(ScrCoef*point_x[now]+2,ScrCoef*point_y[now]+2)], fill = 'black', outline ='black')
  draw.ellipse([(ScrCoef*point_x[start]-4,ScrCoef*point_y[start]-4),(ScrCoef*point_x[start]+4,ScrCoef*point_y[start]+4)], fill = 'blue', outline ='blue')
  draw.ellipse([(ScrCoef*point_x[end]-4,ScrCoef*point_y[end]-4),(ScrCoef*point_x[end]+4,ScrCoef*point_y[end]+4)], fill = 'red', outline ='red')
  for now in range(nCity):
    draw.text((ScrCoef*point_x[now]+2,ScrCoef*point_y[now]+2), str(now),fill='black')
  
  del draw
  name=FoldName+"/"+"time"+str(time)+"best"+str(bestLen)+".png"
  image.save(name, "PNG")

In [ ]:
def probability(i,j): #Расчёт весов определённого отрезка пути для расчёта вероятности прохода муравья по нему.
  p=(pherTrail[i][j]**Alpha)*((1/lenTrail[i][j])**Beta)
  return p

In [ ]:
def UpdPher(): # Обновление феромонов
  for i in range(nCity):
    for j in range (i):
      if(lenTrail[i][j]!=0):
        pherTrail[i][j]=pherTrail[i][j]*(1-Rho)+deltaPher[i][j]
        if(pherTrail[i][j]<pherMin):
          pherTrail[i][j]=pherMin
        if(pherTrail[i][j]>pherMax):
          pherTrail[i][j]=pherMax
        pherTrail[j][i]=pherTrail[i][j]
        deltaPher[i][j]=0
        deltaPher[j][i]=0

In [ ]:
def Move(now,tabu,to): #Функция выбора следующей точки для перехода
  WhereList=[]
  ProbList=[]
  if(len(tabu)==nCity-1):
    return to
  for cit in range(nCity):
    flag=0
    for i in range(len(tabu)):
      if(tabu[i]==cit):
        flag=1
        break
    if(len(tabu)<nCity-1 and cit!=to and flag==0):
      WhereList.append(cit)
      ProbList.append(probability(now,cit))
  return random.choices(WhereList, weights=ProbList)[0]

In [ ]:
def March(Fr,To): #Функция нахождения маршрута муравья из одной точки в другую
  now=Fr
  tabu=[]
  tabu.append(now)
  while 1:
    next=Move(now,tabu,To)
    now=next
    tabu.append(now)
    if (now==To):
      return tabu

In [ ]:
def Length(Trail): #Расчёт длины пути на графе
  l=0
  for i in range(len(Trail)-1):
    l+=lenTrail[Trail[i]][Trail[i+1]]
  return l

In [ ]:
def PutPher(Trail,l): #Распределение феромонов после движения муравьёв
  for sch in range(len(Trail)-1):
    deltaPher[Trail[sch]][Trail[sch+1]]+=Q/l
    deltaPher[Trail[sch+1]][Trail[sch]]=deltaPher[Trail[sch]][Trail[sch+1]]

In [ ]:
def Gross(tabuNow,now): # Функция полного перебора
  if (len(tabuNow)==nCity-1):
    tabuCopy=tabuNow.copy()
    tabuCopy.append(end)
    l=Length(tabuCopy)
    if(len(GrossLen)==0):
      GrossLen.append(round(l,1))
      NowTime=time.time()
      GrossTime.append(round(NowTime-StartTime,4))
      return
    if(l<GrossLen[len(GrossLen)-1]):
      GrossLen.append(round(l,1))
      NowTime=time.time()
      GrossTime.append(round(NowTime-StartTime,4))
      return
  for i in range(nCity):
    flag=0
    if (i==end):
      flag=1
    for j in range(len(tabuNow)):
      if (i==tabuNow[j]):
        flag=1
    if(flag==0):
      tabuCopy=tabuNow.copy()
      tabuCopy.append(i)
      Gross(tabuCopy,i)

In [ ]:
def Together(route1,route2): # Скрещивание для Генетического алгоритма
  newroute=[]
  newrouteTail=[]
  for i in range(len(route1)):
    if (i%2==0):
      for First in range(len(route1)):
        flag=0
        for Finding1 in range(len(newroute)):
          if(newroute[Finding1]==route1[First]):
            flag=1
            break
        if(flag==0):
          for Finding2 in range(len(newrouteTail)):
            if(newrouteTail[Finding2]==route1[First]):
              flag=1
              break
        if(flag==0):
          newroute.append(route1[First])
          break
    else:
      for Second in range(len(route2)):
        flag=0
        for Finding1 in range(len(newroute)):
          if(newroute[Finding1]==route2[len(route2)-Second-1]):
            flag=1
            break
        if(flag==0):
          for Finding2 in range(len(newrouteTail)):
            if(newrouteTail[Finding2]==route2[len(route2)-Second-1]):
              flag=1
              break
        if(flag==0):
          newrouteTail.append(route2[len(route2)-Second-1])
          break
  for i in range(len(newrouteTail)):
    newroute.append(newrouteTail[len(newrouteTail)-i-1])
  return newroute

In [ ]:
def Mutation1(route): #Вариант 1 мутации генома генетичкого алгоритма
  newroute=route.copy()
  cit1=random.randint(1,len(route)-2)
  cit2=random.randint(1,len(route)-2)
  newroute[cit1]=route[cit2]
  newroute[cit2]=route[cit1]
  return newroute

In [ ]:
def Mutation2(route): #Вариант 2 мутации генома генетичкого алгоритма
  newroute=route.copy()
  cit1=random.randint(1,len(route)-2)
  cit2=random.randint(1,len(route)-2)
  delta=cit2-cit1
  if (cit1>cit2):
    for i in range(cit1-cit2+1):
      newroute[cit2+i]=route[cit1-i]
  else:
    for i in range(cit2-cit1+1):
      newroute[cit1+i]=route[cit2-i]
  return newroute

In [ ]:
def Mutation3(route): #Вариант 3 мутации генома генетичкого алгоритма
  newroute=route.copy()
  cit1=random.randint(1,len(route)-2)
  cit2=random.randint(1,len(route)-2)
  delta=cit2-cit1
  if (cit1>cit2):
    for i in range(cit1-cit2):
      newroute[cit2+i+1]=route[cit2+i]
    newroute[cit2]=route[cit1]
  else:
    for i in range(cit2-cit1):
      newroute[cit1+i+1]=route[cit1+i]
    newroute[cit1]=route[cit2]
  return newroute

In [ ]:
def RandGen(): #Случайная генерация первого поколения для генетичкого алгоритма
  newroute=[]
  newroute.append(start)
  inside=[]
  for i in range(nCity):
    if(i!=start and i!=end):
      inside.append(i)
  random.shuffle(inside)
  for j in range(len(inside)):
    newroute.append(inside[j])
  newroute.append(end)
  return newroute

In [ ]:
def Find2Best(genlen): #Нахождение двух лучших геномов среди поколения генетического алгоритма
  top1=100000000
  top2=100000000
  ntop1=0
  ntop2=0
  flagN=0
  for i in range (len(genlen)):
    if (genlen[i]<top1):
      if(flagN==1):
        ntop2=ntop1
      top2=top1
      top1=genlen[i]
      ntop1=i
    elif (genlen[i]<top2):
      flagN=1
      top2=genlen[i]
      ntop2=i
  return ntop1,ntop2

In [ ]:
def FindBest(genlen): #Нахождение одного лучшего генома среди поколения генетического алгоритма
  top1=100000000
  ntop1=0
  for i in range (len(genlen)):
    if (genlen[i]<top1):
      top1=genlen[i]
      ntop1=i
  return ntop1

In [ ]:
def Selection(r1,r2): #Скрещивание двух лучших геномов поколения генетического алгоритма
  child1=Together(r1,r2)
  child2=Together(r2,r1)
  toplist=[r1,r2,child1,child2]
  toplen=[Length(r1),Length(r2),Length(child1),Length(child2)]
  return toplist[FindBest(toplen)]

In [ ]:
def RankModSort(FirstList): #Сортировка списка маршрутов для модификации ASrank
  TopList=[]
  LenList=[]
  for i in range (len(FirstList)-1):
    flag=0
    l=Length(FirstList[i])
    for j in range(len(TopList)-1):
      if(l<LenList[j]):
        flag=1
        TopList.insert(j,FirstList[i])
        LenList.insert(j,l)
        break
    if(flag==0):
      TopList.append(FirstList[i])
      LenList.append(l)
  return TopList, LenList

In [ ]:
def RankModPher(TopList, LenList): #Распределение феромонов для модификации ASrank
  for i in range(len(TopList)-1):
    for sch in range(len(TopList[i])-1):
      deltaPher[TopList[i][sch]][TopList[i][sch+1]]+=(Q/l)*(1-i/len(TopList))
      deltaPher[TopList[i][sch+1]][TopList[i][sch]]=deltaPher[TopList[i][sch]][TopList[i][sch+1]]

In [ ]:
def LocalFind(route): #Гибридный элемент локального поиска на маршруте
  best=Length(route)
  outroute=route.copy()
  for i in range(2,len(route)-2):
    for j in range(i-10,i):
      if(j<1):
        break
      newroute=route.copy()
      for k in range(i-j):
        newroute[j+k+1]=route[j+k]
      newroute[j]=route[i]
      if (Length(newroute)<best):
        best=Length(newroute)
        outroute=newroute
  return outroute

In [ ]:
point_x=[]
point_y=[]
point_near=[]

window_x=400 #ширина поля
window_y=400 #высота поля
ScrCoef=2 #Масштабирование при выводе карт

start=0 #Точка муравейника
end=1 #Точка цели

nCity=50 #Количество вершин

lenTrail = [[0 for j in range(nCity)] for i in range(nCity)]
do_point_map()
FirstDraw() #Сохранение рисунка карты


In [ ]:
nAnt=25 #Количество простых муравьёв
Alpha=2 #Коэффициент альфа (порядка значимости феромона)
Beta=4 #Коэффициент бэта (порядка значимости длины пути)
Rho=0.3 #Коэффициент испарения феромона
Q=2000 #Коэффициент увеличения феромона

pherMin=1 #Минимальное количество феромона на рёбрах графа
pherMax=30 #Максимальное количество феромона на рёбрах графа

In [ ]:
# Модификации
nElite=0 #Количество элитных муравьёв
ASrank=0 #Модификация ASrank. 1 - активно, 0 - неактивно
LocalMin=1 #Добавление элемента локального поиска. 1 - активно, 0 - неактивно

In [ ]:
# Имена и циклы
dir_name="Tests"

# Первая переменная цикла 
Par1Name="Q"
Rng1max = 1
Par1Mas=[500,1000,2000,4000,8000,12000,16000] # массив значений 1ой переменной

# Вторая переменная цикла
Par2Name="Rho"
Rng2max = 1
Par2Mas=[0.3] # массив значений 2ой переменной

# Третья переменная цикла
Par3Name="nElite"
Rng3max = 1
Par3Mas=[0,1,2,3,4,5,7,10]

MaxCikl=9

ShowProcessMaps=1 # Отображение промежуточных маршрутов на картах: 0 - нет, 1 - да
ShowProgressString=0 # Отображение строк с нахождением новых оптимальных путей: 0 - нет, 1 - да
AllLines=0 # Отображение на промежуточных картах всех рёбер и феромонов: 0 - нет, 1 - да

In [ ]:
#Цикл для исследования ACO с различными параметрами
if (os.path.exists(dir_name)):
  rmtree(dir_name)
if (ShowProcessMaps==1):
  os.mkdir(dir_name)

BestList=[]
TimerList=[]
StepList=[]
MedianList=[]
MidList=[]
TimeMidList=[]

for Par1 in range(Rng1max):
  #Q=Par1Mas[Par1]
  
  BestListPar1=[]
  TimerListPar1=[]
  StepListPar1=[]
  MedianListPar1=[]
  MidListPar1=[]
  TimeMidListPar1=[]
  
  if(ShowProcessMaps==1):
    dir_name1=dir_name+"/"+Par1Name+"_"+str(Par1)
    os.mkdir(dir_name1)
  for Par2 in range(Rng2max):
    #Rho=Par2Mas[Par2]

    BestListPar2=[]
    TimerListPar2=[]
    StepListPar2=[]
    MedianListPar2=[]
    MidListPar2=[]
    TimeMidListPar2=[]

    if(ShowProcessMaps==1):
      dir_name2=dir_name1+"/"+Par2Name+"_"+str(Par2)
      os.mkdir(dir_name2)

    for Par3 in range(Rng3max):
      #Q=Par3Mas[Par3]

      BestListPar3=[]
      TimerListPar3=[]
      StepListPar3=[]

      if(ShowProcessMaps==1):
        dir_name3=dir_name2+"/"+Par3Name+"_"+str(Par3)
        os.mkdir(dir_name3)
      for Cikl in range(MaxCikl):
        BestListCikl=[]
        TimerListCikl=[]
        StepListCikl=[]
        RealStartTime = time.time()

        if(ShowProcessMaps==1):
          dir_name4=dir_name3+"/"+"Cikl"+"_"+str(Cikl+1)
          os.mkdir(dir_name4)
        pherTrail = [[pherMin for j in range(nCity)] for i in range(nCity)]
        deltaPher = [[0 for j in range(nCity)] for i in range(nCity)]
        bestLen=-1
        bestTrail=[]
        bestTime=0
        #Tend//nAnt+(nAnt//10)*70
        step=0
        while(time.time()-RealStartTime<1.5):
          step+=1

          if(ASrank==1):
            StepRoutes=[]
            StepLens=[]

          for Ant in range(nAnt):
            if(step%2==0):
              way=March(start,end)
            else:
              way=March(end,start)
            if(LocalMin==1):
              way=LocalFind(way)
            l=Length(way)
            if(bestLen==-1 or l<bestLen):
              bestLen=l
              bestTrail=way
              bestTime=step
              
              RealNowTime = time.time()
              BestListCikl.append(bestLen)
              TimerListCikl.append(RealNowTime-RealStartTime)
              StepListCikl.append(step)

              if(ShowProgressString==1):
                print("На шаге",step," (",round(RealNowTime-RealStartTime,4),"сек) найден путь длиной",bestLen)
              if(ShowProcessMaps==1):
                ProgressDraw(step,round(bestLen,1),bestTrail,dir_name4)
            
              if(ASrank==1):
                StepRoutes.append(way)
            if(ASrank==1):
              StepRoutes,StepLens=RankModSort(StepRoutes)
              RankModPher(StepRoutes,StepLens)
            else:
              PutPher(way,l)

          for EliteAnt in range(nElite):
            PutPher(bestTrail,bestLen)
          UpdPher()
        if(ShowProgressString==1):
          print("len",bestLen)
          print("step",bestTime)
          print("time",round(RealNowTime-RealStartTime,4))
          #print(Par1Name,Par1Mas[Par1])
          #print(Par2Name,Par2Mas[Par2])
          #print(Par3Name,Par3Mas[Par3])
          #print("Cikl = "+str(Cikl))
        print(Par1*Rng2max*Rng3max*MaxCikl+Par2*Rng3max*MaxCikl+Par3*MaxCikl+Cikl+1,"/",Rng1max*Rng2max*Rng3max*MaxCikl)

        BestListPar3.append(BestListCikl) #Собираем в список с каждого цикла
        TimerListPar3.append(TimerListCikl)
        StepListPar3.append(StepListCikl)
      
      copyBest=BestListPar3.copy()
      copyBest.sort()
      Median=copyBest[MaxCikl//2]
      for k in range (MaxCikl):
        if(BestListPar3[k]==Median):
          MedianNum=k
          break
      
      sum=0
      for k in range (MaxCikl):
        sum+=BestListPar3[k][len(BestListPar3[k])-1]
      Mid=sum/MaxCikl
      MidListPar2.append(Mid)

      Timesum=0
      for k in range (MaxCikl):
        Timesum+=TimerListPar3[k][len(BestListPar3[k])-1]
      TimeMid=Timesum/MaxCikl
      TimeMidListPar2.append(TimeMid)


      MedianListPar2.append(MedianNum) #Собираем в список разных параметров 3
      BestListPar2.append(BestListPar3) 
      TimerListPar2.append(TimerListPar3)
      StepListPar2.append(StepListPar3)
    MidListPar1.append(MidListPar2)
    TimeMidListPar1.append(TimeMidListPar2)
    MedianListPar1.append(MedianListPar2) #Собираем в список разных параметров 2
    BestListPar1.append(BestListPar2) 
    TimerListPar1.append(TimerListPar2)
    StepListPar1.append(StepListPar2)
  MidList.append(MidListPar1)
  TimeMidList.append(TimeMidListPar1)
  MedianList.append(MedianListPar1) #Собираем в список разных параметров 1
  BestList.append(BestListPar1)
  TimerList.append(TimerListPar1)
  StepList.append(StepListPar1)


1 / 9
2 / 9
3 / 9
4 / 9
5 / 9
6 / 9
7 / 9
8 / 9
9 / 9


In [ ]:
#Полный перебор
StartTime = time.time()
tabu=[]
GrossTime=[]
GrossLen=[]
tabu.append(start)
Gross(tabu,start)
NowTime=time.time()
AllTime=NowTime-StartTime
print(AllTime)
print(GrossLen[len(GrossLen)-1])

KeyboardInterrupt: ignored

In [ ]:
#Генетический алгоритм
nGen=50 #Кол-во особей в поколении
Epoch=1000 #Кол-во поколений
RealStartTime = time.time()
nowgen=[]
nowgenlen=[]
GenLen=[]
GenTime=[]
for i in range (nGen):
  nowroute=RandGen()
  nowgen.append(nowroute)
  nowgenlen.append(Length(nowroute))
top1,top2=Find2Best(nowgenlen)

for step in range(Epoch):

  Leader=Selection(nowgen[top1],nowgen[top2])

  for i in range(nGen-1):
    mark=random.randint(1,3)
    if(mark==1):
      nowgen[i]=Mutation1(Leader)
    elif(mark==2):
      nowgen[i]=Mutation2(Leader)
    else:
      nowgen[i]=Mutation3(Leader)
    nowgenlen[i]=Length(nowgen[i])
  nowgen[nGen-1]=Leader
  nowgenlen[nGen-1]=Length(Leader)
  top1,top2=Find2Best(nowgenlen)
  if(len(GenLen)==0):
    GenLen.append(nowgenlen[top1])
    RealNowTime = time.time()
    GenTime.append(RealNowTime-RealStartTime)
  elif(GenLen[len(GenLen)-1]>nowgenlen[top1]):
    GenLen.append(nowgenlen[top1])
    RealNowTime = time.time()
    GenTime.append(RealNowTime-RealStartTime)
print(GenLen[len(GenLen)-1])
print(GenTime[len(GenTime)-1])

2526.421302129419
1.8947572708129883


In [ ]:
#ACO. Cохранение истории нахождения оптимального маршрута в Excel.
wb = openpyxl.Workbook()
wb.create_sheet(title = 'History', index = 0)
sheet = wb['History']

for i in range (len(BestList)):
  for j in range (len(BestList[i])):
    for k in range (len(BestList[i][j])):
      NowColumn=i*len(BestList[i])*len(BestList[i][j])*5+j*len(BestList[i][j])*5+k*5+1
      NowCell = sheet.cell(row = 1, column = NowColumn)
      NowCell.value = Par1Name+str(Par1Mas[i])
      NowCell = sheet.cell(row = 2, column = NowColumn)
      NowCell.value = Par2Name+str(Par2Mas[j])
      NowCell = sheet.cell(row = 3, column = NowColumn)
      NowCell.value = Par3Name+str(Par3Mas[k])
      NowCell = sheet.cell(row = 4, column = NowColumn)
      b1=round(BestList[i][j][k][MedianList[i][j][k]][len(BestList[i][j][k][MedianList[i][j][k]])-1],1)
      NowCell.value = "Best="+str(b1)
      
      NowColumn=i*len(BestList[i])*len(BestList[i][j])*5+j*len(BestList[i][j])*5+k*5+2
      NowCell = sheet.cell(row = 1, column = NowColumn)
      NowCell.value = "Best"
      
      for Sch1 in range (len(BestList[i][j][k][MedianList[i][j][k]])):
        NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
        NowCell.value =round(BestList[i][j][k][MedianList[i][j][k]][Sch1],1)

      NowColumn=i*len(BestList[i])*len(BestList[i][j])*5+j*len(BestList[i][j])*5+k*5+3
      NowCell = sheet.cell(row = 1, column = NowColumn)
      NowCell.value = "Step"
      for Sch1 in range (len(StepList[i][j][k][MedianList[i][j][k]])):
        NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
        NowCell.value =round(StepList[i][j][k][MedianList[i][j][k]][Sch1])

      NowColumn=i*len(BestList[i])*len(BestList[i][j])*5+j*len(BestList[i][j])*5+k*5+4
      NowCell = sheet.cell(row = 1, column = NowColumn)
      NowCell.value = "Time"
      for Sch1 in range (len(TimerList[i][j][k][MedianList[i][j][k]])):
        NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
        NowCell.value =round(TimerList[i][j][k][MedianList[i][j][k]][Sch1],4)

wb.save('AntHistory.xlsx')

In [ ]:
#ACO. Cохранение средних длин найденных оптимальных маршрутов.
wb = openpyxl.Workbook()
wb.create_sheet(title = 'TimeBest', index = 0)
sheet2 = wb['TimeBest']
wb.create_sheet(title = 'Best', index = 0)
sheet = wb['Best']

for i in range (Rng2max):
  NowColumn=i*(Rng3max+2)+1
  NowCell = sheet.cell(row = 1, column = NowColumn)
  NowCell.value = Par2Name+"="+str(Par2Mas[i])
  for sch in range (Rng1max):
    NowCell = sheet.cell(row = sch+2, column = NowColumn)
    NowCell.value = Par1Name+"="+str(Par1Mas[sch])
  for j in range (Rng3max):
    NowColumn=i*(Rng3max+2)+j+2
    NowCell = sheet.cell(row = 1, column = NowColumn)
    NowCell.value = Par3Name+"="+str(Par3Mas[j])
    for k in range (Rng1max):
      NowCell = sheet.cell(row = k+2, column = NowColumn)
      NowCell.value = round(MidList[k][i][j],1)

for i in range (Rng2max):
  NowColumn=i*(Rng3max+2)+1
  NowCell = sheet2.cell(row = 1, column = NowColumn)
  NowCell.value = Par2Name+"="+str(Par2Mas[i])
  for sch in range (Rng1max):
    NowCell = sheet2.cell(row = sch+2, column = NowColumn)
    NowCell.value = Par1Name+"="+str(Par1Mas[sch])
  for j in range (Rng3max):
    NowColumn=i*(Rng3max+2)+j+2
    NowCell = sheet2.cell(row = 1, column = NowColumn)
    NowCell.value = Par3Name+"="+str(Par3Mas[j])
    for k in range (Rng1max):
      NowCell = sheet2.cell(row = k+2, column = NowColumn)
      NowCell.value = round(TimeMidList[k][i][j],4)
      
wb.save('AntSheet.xlsx')

In [ ]:
#Метод перебора. Cохранение истории нахождения оптимального маршрута в Excel.
Grwb = openpyxl.Workbook()
Grwb.create_sheet(title = 'Первый лист', index = 0)
sheet = Grwb['Первый лист']
  
NowColumn=1
NowCell = sheet.cell(row = 1, column = NowColumn)
b1=GrossLen[len(GrossLen)-1]
NowCell.value = "Best="+str(b1)
NowCell = sheet.cell(row = 2, column = NowColumn)
NowCell.value = "Time="+str(AllTime)
NowColumn=2
NowCell = sheet.cell(row = 1, column = NowColumn)
NowCell.value = "Best"
for Sch1 in range (len(GrossLen)):
  NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
  NowCell.value =GrossLen[Sch1]
NowColumn=3
NowCell = sheet.cell(row = 1, column = NowColumn)
NowCell.value = "Time"
for Sch1 in range (len(GrossTime)):
  NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
  NowCell.value =GrossTime[Sch1]

Grwb.save('Gross.xlsx')

In [ ]:
#Генетический алгоритм. Cохранение истории нахождения оптимального маршрута в Excel.
Genwb = openpyxl.Workbook()
Genwb.create_sheet(title = 'Первый лист', index = 0)
sheet = Genwb['Первый лист']

NowColumn=1
NowCell = sheet.cell(row = 1, column = NowColumn)
b1=GenLen[len(GenLen)-1]
NowCell.value = "Best="+str(b1)
NowColumn=2
NowCell = sheet.cell(row = 1, column = NowColumn)
NowCell.value = "Best"
for Sch1 in range (len(GenLen)):
  NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
  NowCell.value =GenLen[Sch1]
NowColumn=3
NowCell = sheet.cell(row = 1, column = NowColumn)
NowCell.value = "Time"
for Sch1 in range (len(GenTime)):
  NowCell = sheet.cell(row = Sch1+2, column = NowColumn)
  NowCell.value =GenTime[Sch1]

Genwb.save('Gen.xlsx')